# 1. Crawl data

In [6]:
import pandas as pd
from selenium import webdriver
import time
cd = input('enter directory:')

def crawl(driverfile = cd+'/chromedriver', url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'):
    driver = webdriver.Chrome(driverfile)
    driver.get(url)
    time.sleep(2)
    listneighbor = driver.find_elements_by_xpath('//table[@class="wikitable sortable jquery-tablesorter"]/tbody/tr')

    neighbor = pd.DataFrame(columns=['Postal Code','Borough','Neighborhood'])
    for i in range(len(listneighbor)):
        neighbor.loc[i,'Postal Code'] = eval(f'driver.find_element_by_xpath(\'//table[@class="wikitable sortable jquery-tablesorter"]/tbody/tr[{i+1}]/td[1]\').text')
        neighbor.loc[i,'Borough'] = eval(f'driver.find_element_by_xpath(\'//table[@class="wikitable sortable jquery-tablesorter"]/tbody/tr[{i+1}]/td[2]\').text')
        neighbor.loc[i,'Neighborhood'] = eval(f'driver.find_element_by_xpath(\'//table[@class="wikitable sortable jquery-tablesorter"]/tbody/tr[{i+1}]/td[3]\').text')
    return neighbor

In [7]:
neighbor = crawl()
neighbor = neighbor[neighbor.Borough != 'Not assigned']
neighbor.Neighborhood.replace('Not assigned',neighbor.Borough, inplace = True)
print('Shape: ',neighbor.shape)

Shape:  (103, 3)


# 2. Location of neighborhoods

In [8]:
latlng = pd.read_csv(cd+'/Geospatial_Coordinates.csv')
neighbor = neighbor.merge(latlng, how='left', on = 'Postal Code')
neighbor.head()


,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


# 3. Clustering


In [21]:
from geopy.geocoders import Nominatim
import folium

address = 'Toronto, ON'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))


map_toronto = folium.Map(location = [latitude, longitude], zoom_start = 10)

for lat, lng, borough, neighborhood in zip(neighbor['Latitude'],neighbor['Longitude'],neighbor['Borough'],neighbor['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius= 5,
        popup=label,
        color = 'blue',
        fill = True,
        fill_color = '#3186cc',
        fill_opacity = 0.7,
        parse_html = False).add_to(map_toronto)
map_toronto 
#hsdfh

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.
